In [8]:
# essential libraries
import json
import random
from urllib.request import urlopen

# storing and analysis

import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import calmap
import folium

# color pallette
cnf = '#393e46' # confirmed - grey
dth = '#ff2e63' # death - red
rec = '#21bf73' # recovered - cyan
act = '#fe9801' # active case - yellow

# converter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# hide warnings
import warnings
warnings.filterwarnings('ignore')

# html embedding
from IPython.display import Javascript
from IPython.core.display import display
from IPython.core.display import HTML

In [21]:
full_table=pd.read_csv('covid_19_clean_complete.csv',parse_dates=['Date'])
full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Thailand,15.0000,101.0000,2020-01-22,2,0,0
1,NaN,Japan,36.0000,138.0000,2020-01-22,2,0,0
2,NaN,Singapore,1.2833,103.8333,2020-01-22,0,0,0
3,NaN,Nepal,28.1667,84.2500,2020-01-22,0,0,0
4,NaN,Malaysia,2.5000,112.5000,2020-01-22,0,0,0


In [22]:
full_table.shape

(11960, 8)

In [38]:
# checking any null values
print(full_table.info())
print(full_table.isna().any())
print(full_table.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11960 entries, 0 to 11959
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province/State  5876 non-null   object        
 1   Country/Region  11960 non-null  object        
 2   Lat             11960 non-null  float64       
 3   Long            11960 non-null  float64       
 4   Date            11960 non-null  datetime64[ns]
 5   Confirmed       11960 non-null  int64         
 6   Deaths          11960 non-null  int64         
 7   Recovered       11960 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 747.6+ KB
None
Province/State     True
Country/Region    False
Lat               False
Long              False
Date              False
Confirmed         False
Deaths            False
Recovered         False
dtype: bool
Province/State    6084
Country/Region       0
Lat                  0
Long       

In [40]:
cases=['Confirmed','Deaths','Recovered','Active']

In [45]:
# active case=confirmed-death-recovered
full_table['Active']=full_table['Confirmed']-full_table['Deaths']\
                    -full_table['Recovered']
full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,NaN,Thailand,15.0000,101.0000,2020-01-22,2,0,0,2
1,NaN,Japan,36.0000,138.0000,2020-01-22,2,0,0,2
2,NaN,Singapore,1.2833,103.8333,2020-01-22,0,0,0,0
3,NaN,Nepal,28.1667,84.2500,2020-01-22,0,0,0,0
4,NaN,Malaysia,2.5000,112.5000,2020-01-22,0,0,0,0


In [46]:
full_table['Country/Region']=full_table['Country/Region']\
            .replace('Mainland China','China')

In [49]:
# list of all countries
full_table['Country/Region'].unique()

array(['Thailand', 'Japan', 'Singapore', 'Nepal', 'Malaysia', 'Canada',
       'Australia', 'Cambodia', 'Sri Lanka', 'Germany', 'Finland',
       'United Arab Emirates', 'Philippines', 'India', 'Italy', 'Sweden',
       'Spain', 'Belgium', 'Egypt', 'Lebanon', 'Iraq', 'Oman',
       'Afghanistan', 'Bahrain', 'Kuwait', 'Algeria', 'Croatia',
       'Switzerland', 'Austria', 'Israel', 'Pakistan', 'Brazil',
       'Georgia', 'Greece', 'North Macedonia', 'Norway', 'Romania',
       'Estonia', 'Netherlands', 'San Marino', 'Belarus', 'Iceland',
       'Lithuania', 'Mexico', 'New Zealand', 'Nigeria', 'Ireland',
       'Luxembourg', 'Monaco', 'Qatar', 'Ecuador', 'Azerbaijan',
       'Armenia', 'Dominican Republic', 'Indonesia', 'Portugal',
       'Andorra', 'Latvia', 'Morocco', 'Saudi Arabia', 'Senegal',
       'Argentina', 'Chile', 'Jordan', 'Ukraine', 'Hungary',
       'Liechtenstein', 'Poland', 'Tunisia', 'Bosnia and Herzegovina',
       'Slovenia', 'South Africa', 'Bhutan', 'Cameroon', 'Colo

In [58]:
# handling missing values
full_table[['Province/State']]=full_table[['Province/State']]\
.fillna('')
full_table[cases]=full_table[cases].fillna(0)


In [62]:
full_table.isnull().any()

Province/State    False
Country/Region    False
Lat               False
Long              False
Date              False
Confirmed         False
Deaths            False
Recovered         False
Active            False
dtype: bool

In [65]:
# cases in the ships
ship=full_table[full_table['Province/State'].str.contains('Grand Princess')|full_table['Country/Region'].str.contains('Cruise Ship')]
ship

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
105,Grand Princess,US,37.6489,-122.6655,2020-01-22,0,0,0,0
167,Diamond Princess,Cruise Ship,35.4437,139.6380,2020-01-22,0,0,0,0
226,Grand Princess,Canada,37.6489,-122.6655,2020-01-22,0,0,0,0
335,Grand Princess,US,37.6489,-122.6655,2020-01-23,0,0,0,0
397,Diamond Princess,Cruise Ship,35.4437,139.6380,2020-01-23,0,0,0,0
...,...,...,...,...,...,...,...,...,...
11667,Diamond Princess,Cruise Ship,35.4437,139.6380,2020-03-12,696,7,325,364
11726,Grand Princess,Canada,37.6489,-122.6655,2020-03-12,0,0,0,0
11835,Grand Princess,US,37.6489,-122.6655,2020-03-13,21,0,0,21
11897,Diamond Princess,Cruise Ship,35.4437,139.6380,2020-03-13,696,7,325,364


In [69]:
# china and the row
china=full_table[full_table['Country/Region']=='China']
row=full_table[full_table['Country/Region']!='China']

In [73]:
# latest cases
full_latest=full_table[full_table['Date']==max(full_table['Date'])].reset_index()
china_latest=full_latest[full_latest['Country/Region']=='China']
row_latest=full_latest[full_latest['Country/Region']!='China']
full_latest.head()

,index,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,11730,,Thailand,15.0000,101.0000,2020-03-13,75,1,35,39
1,11731,,Japan,36.0000,138.0000,2020-03-13,701,19,118,564
2,11732,,Singapore,1.2833,103.8333,2020-03-13,200,0,97,103
3,11733,,Nepal,28.1667,84.2500,2020-03-13,1,0,1,0
4,11734,,Malaysia,2.5000,112.5000,2020-03-13,197,0,26,171


In [86]:
# latest condensed
full_latest_grouped=full_latest.groupby('Country/Region')[cases].sum().reset_index()
china_latest_grouped=china_latest.groupby('Province/State')[cases].sum().reset_index()
row_latest_grouped=row_latest.groupby('Country/Region')[cases].sum().reset_index()
row_latest_grouped.head(),full_latest_grouped.head(),china_latest_grouped.head()

(        Country/Region  Confirmed  Deaths  Recovered  Active
 0          Afghanistan          7       0          0       7
 1              Albania         33       1          0      32
 2              Algeria         26       2          8      16
 3              Andorra          1       0          0       1
 4  Antigua and Barbuda          1       0          0       1,
         Country/Region  Confirmed  Deaths  Recovered  Active
 0          Afghanistan          7       0          0       7
 1              Albania         33       1          0      32
 2              Algeria         26       2          8      16
 3              Andorra          1       0          0       1
 4  Antigua and Barbuda          1       0          0       1,
   Province/State  Confirmed  Deaths  Recovered  Active
 0          Anhui        990       6        984       0
 1        Beijing        436       8        342      86
 2      Chongqing        576       6        566       4
 3         Fujian        296  

In [89]:
# Latest Complete Data
temp=full_table.groupby(['Country/Region','Province/State'])[cases].max()
temp.head()

,,Confirmed,Deaths,Recovered,Active
Country/Region,Province/State,,,,
Afghanistan,,7,0,0,7
Albania,,33,1,0,32
Algeria,,26,2,8,20
Andorra,,1,0,1,1
Antigua and Barbuda,,1,0,0,1


In [113]:
# latest condensed data
temp=full_table.groupby('Date')[cases].sum().reset_index()
temp=temp[temp['Date']==max(temp['Date'])].reset_index(drop=True)
temp.style.background_gradient(cmap='Pastel1')

,Date,Confirmed,Deaths,Recovered,Active
0,2020-03-13 00:00:00,145193,5404,70251,69538


In [119]:
tm=temp.melt(id_vars='Date',value_vars=['Active','Deaths','Recovered'])
fig=px.treemap(tm,path=['variable'],values='value',height=400,width=600,
              color_discrete_sequence=[rec,act,dth])
fig.show()

In [121]:
# country wise data In each country